## Importação dos pacotes

In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# importar os pacotes necessários para os algoritmos de regressão
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.ensemble import VotingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

## Carga dos dados de entrada

In [4]:
# carregar arquivo de dados de treino
train_data = pd.read_csv('abalone-train.csv', index_col='id')

In [5]:
# carregar arquivo de dados de teste
test_data = pd.read_csv('abalone-test.csv', index_col='id')

## Transformações nos dados

In [6]:
# gerar "one hot encoding" em atributos categóricos
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [7]:
# encontrar e remover possíveis outliers
data = train_data
outliers = np.concatenate((
    data[(data['height'] < 0.01) | (data['height'] > 0.3)].index,
    data[(data['viscera_weight'] < 0.0001) | (data['viscera_weight'] > 0.6)].index
), axis=0)
train_data.drop(outliers, inplace=True)

## Seleção dos dados de treino e teste

In [8]:
# definir dados de treino

X_train = train_data.drop(['rings'], axis=1) # tudo, exceto a coluna alvo
y_train = train_data['rings'] # apenas a coluna alvo

print('Forma dos dados de treino:', X_train.shape, y_train.shape)

Forma dos dados de treino: (2781, 10) (2781,)


In [9]:
# definir dados de teste

X_test = test_data # tudo, já que não possui a coluna alvo

print('Forma dos dados de teste:', X_test.shape)

Forma dos dados de teste: (1393, 10)


## Treinamento dos modelos e geração dos resultados 

In [10]:
models = []

# Generalized Linear Models
models.append(('LinReg', LinearRegression(n_jobs=2, fit_intercept=False, normalize=True)))
models.append(('LogReg', LogisticRegression(n_jobs=2, random_state=42, multi_class='auto', C=1000, solver='newton-cg')))
models.append(('OMP', OrthogonalMatchingPursuit(n_nonzero_coefs=8, fit_intercept=True, normalize=True)))
models.append(('PAR', PassiveAggressiveRegressor(random_state=42, C=0.1, fit_intercept=True, max_iter=1000, tol=0.001)))
models.append(('PP', Perceptron(random_state=42, penalty='l2', alpha=1e-5, fit_intercept=True, max_iter=1000, tol=1e-3)))
models.append(('RANSAC', RANSACRegressor(random_state=42, min_samples=0.75)))
models.append(('Ridge', Ridge(random_state=42, alpha=0.1, fit_intercept=False, normalize=True)))
models.append(('SGD', SGDRegressor(random_state=42, alpha=1e-06, fit_intercept=True, penalty=None, tol=1e-3)))
models.append(('TSR', TheilSenRegressor(random_state=42, n_jobs=2, fit_intercept=True)))

# Decision Trees
models.append(('DTR', DecisionTreeRegressor(random_state=42, max_depth=4, min_samples_split=0.25)))

# Gaussian Processes
models.append(('GPR', GaussianProcessRegressor(random_state=42, alpha=0.01, normalize_y=True)))

# Kernel Ridge Regression
models.append(('KRR', KernelRidge()))

# Naïve Bayes
models.append(('GNB', GaussianNB(var_smoothing=0.1)))

# Nearest Neighbors
models.append(('kNN', KNeighborsRegressor(n_jobs=2, n_neighbors=13, weights='distance')))

# Support Vector Machines
models.append(('SVM', SVR(gamma='auto', kernel='linear')))

# Neural network models
models.append(('MLP', MLPRegressor(random_state=42, max_iter=500,
                     activation='logistic', hidden_layer_sizes=(50,), solver='lbfgs')))

# Ensemble Methods
models.append(('RFR', RandomForestRegressor(random_state=42, n_jobs=2, n_estimators=100, max_depth=7)))
models.append(('GBR', GradientBoostingRegressor(random_state=42, learning_rate=0.05, n_estimators=100,
                                  subsample=0.4, max_depth=6, max_features=1.0)))
models.append(('ETR', ExtraTreesRegressor(random_state=42, n_jobs=2, n_estimators=200, max_features=1.0)))
models.append(('BDTR', BaggingRegressor(random_state=42, n_jobs=2, base_estimator=DecisionTreeRegressor())))
models.append(('ABDTR', AdaBoostRegressor(random_state=42, n_estimators=100, base_estimator=DecisionTreeRegressor())))

# XGBoost
models.append(('XGBR', XGBRegressor(random_state=42, n_jobs=2, learning_rate=0.1,
                                    n_estimators=50, max_depth=5, objective='reg:squarederror')))

In [11]:
models = []
models.append(('MLP', MLPRegressor(random_state=42, max_iter=500,
                     activation='logistic', hidden_layer_sizes=(50,), solver='lbfgs')))
models.append(('GPR', GaussianProcessRegressor(random_state=42, alpha=0.01, normalize_y=True)))
models.append(('XGBR', XGBRegressor(random_state=42, n_jobs=2, learning_rate=0.1,
                                    n_estimators=50, max_depth=5, objective='reg:squarederror')))

In [10]:
!mkdir submissions

mkdir: cannot create directory ‘submissions’: File exists


In [12]:
sufixo_arquivo = '27jun'

for name, model in models:
    print(model, '\n')
    
    # treinar o modelo
    model.fit(X_train, y_train)
    
    # executar previsão usando o modelo
    y_pred = model.predict(X_test)
    
    # gerar dados de envio (submissão)
    submission = pd.DataFrame({
      'id': X_test.index,
      'rings': y_pred
    })
    submission.set_index('id', inplace=True)

    # gerar arquivo CSV para o envio
    filename = 'submissions/abalone-submission-p-%s-%s.csv' % (sufixo_arquivo, name.lower())
    submission.to_csv(filename)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=42, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False) 

GaussianProcessRegressor(alpha=0.01, copy_X_train=True, kernel=None,
             n_restarts_optimizer=0, normalize_y=True,
             optimizer='fmin_l_bfgs_b', random_state=42) 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=2, nthread=None, objective='reg:squarederror',
       random_state=42, reg_alpha=0

/opt/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [13]:
# verificar conteúdo dos arquivos gerados
!head submissions/abalone-submission-p-*.csv

==> submissions/abalone-submission-p-27jun-gpr.csv <==
id,rings
1512,9.523528170614773
1401,11.078191704907331
509,13.19897035189978
2883,8.422100484357307
3259,11.878710506395
2135,9.217414277158232
1499,9.272504075238194
3597,11.890077604584112
2414,8.058075097065503

==> submissions/abalone-submission-p-27jun-mlp.csv <==
id,rings
1512,9.812937062341494
1401,10.799840547320592
509,13.764178000579594
2883,8.287674228327134
3259,11.324386343077478
2135,9.053522403788886
1499,9.150520578860018
3597,11.538201561477969
2414,8.192894346154812

==> submissions/abalone-submission-p-27jun-xgbr.csv <==
id,rings
1512,10.042686
1401,11.201387
509,11.6318035
2883,7.904088
3259,10.762072
2135,9.135977
1499,9.858093
3597,12.28695
2414,7.916978

==> submissions/abalone-submission-p-abdtr.csv <==
id,rings
1512,10.0
1401,11.0
509,11.0
2883,8.0
3259,11.0
2135,9.0
1499,10.0
3597,11.0
2414,8.0

==> submissions/abalone-submission-p-bdtr.csv <==
id,rings
1512,9.5
1401,10.5
509,11.1
2883,8.4
3259,11.9
2135,

id,rings
1512,10.3196261682243
1401,12.63064833005894
509,12.194174757281553
2883,7.460554371002132
3259,12.63064833005894
2135,9.820564516129032
1499,10.3196261682243
3597,12.63064833005894
2414,7.460554371002132

==> submissions/abalone-submission-p-o4-etr.csv <==
id,rings
1512,10.365
1401,10.53
509,10.66
2883,8.47
3259,11.43
2135,8.675
1499,9.84
3597,11.775
2414,8.15

==> submissions/abalone-submission-p-o4-gbr.csv <==
id,rings
1512,10.16856920474763
1401,11.267367936361739
509,11.126073262885072
2883,8.087839938142217
3259,10.747164098912076
2135,9.059914612569791
1499,9.751131252839723
3597,11.641050337940516
2414,7.9107575965198365

==> submissions/abalone-submission-p-o4-gnb.csv <==
id,rings
1512,11
1401,11
509,8
2883,7
3259,11
2135,9
1499,10
3597,11
2414,6

==> submissions/abalone-submission-p-o4-gpr.csv <==
id,rings
1512,9.355318978028961
1401,11.063983009922666
509,13.417888380304547
2883,8.363015852088964
3259,11.895245616316586
2135,9.145761142158406
1499,9.281954381198219


id,rings
1512,9.450294320761408
1401,11.0727681606621
509,13.226506458349014
2883,8.353146624774446
3259,11.876225907909976
2135,9.19481535748421
1499,9.29027344072356
3597,11.88841858051942
2414,7.973501039081931

==> submissions/abalone-submission-p-o9-0.50-knn.csv <==
id,rings
1512,10.450920122076752
1401,10.547940060237595
509,11.000246426623573
2883,8.294157251191635
3259,12.770812891655028
2135,8.334921659332283
1499,9.948263505915676
3597,11.613391575547311
2414,7.302274816638973

==> submissions/abalone-submission-p-o9-0.50-krr.csv <==
id,rings
1512,9.260158740431958
1401,11.398415200789806
509,11.311362045194635
2883,8.43195008261712
3259,11.971832105552643
2135,9.786516564120603
1499,9.962333500586581
3597,12.803469702700205
2414,8.171490843022305

==> submissions/abalone-submission-p-o9-0.50-linreg.csv <==
id,rings
1512,8.837508170117534
1401,11.24050033702234
509,11.567415415345351
2883,8.378262092796355
3259,11.946968808208988
2135,9.203234696435807
1499,9.459511877320914
